# Задача коммивояжёра на реальных картах

In [ ]:
!pip install -r requirements.txt

Доставка товаров - неотъемлемая часть любого бизнес-процесса в реальном секторе экономики. Чтобы снизить себестоимость и цену на услуги для потребителя, заказы объединяют и доставляются одним курьером. Здесь появляется так называемый эффект масштаба, когда себестоимость доставки единицы заказа снижается с увеличением их количества.

В "тривиальном" случае, когда планирование идет на уровне одного курьера и пула заказов, возникает классическая задача коммивояжера (нетривиальный известен как vehicle routing problem). **Суть задачи** - найти оптимальную последовательность точек доставки, которая минимизировала бы логистические издержки выражаемые главным образом во времени доставки. Нетривиальность заключается в том, что сложность пространства решения растет экспоненциально с числом точек доставки, тем самым делая вычислительно невозможным поиск точного решения.\
Так, например, для 86000 точек доставки вычисление точного решения методом ветвей и границ составило 136 ЦПУ-лет (1 ЦПУ-час = час работы процессора с производительностью 1 GFLOPs, 1 A100 = 20 TFLOPS, кластер Сбера Кристофари NEO = 792 A100 $\approx$ 16 GFLOPS)

Поскольку в реальной жизни слишком много факторов, влияющих на время доставки, задачу коммивояжера решают быстрыми приближенными алгоритмами, эффективность которых при правильной настройке уступает оптимальному решению всего лишь на 3-4%. 

## Загрузка карты и задачи

В этом задании мы рассмотрим задачу коммивояжера в контексте города Казань (Москва и Санкт-Петербург слишком большие, чтобы вычислять без оптимизаций компилятора).\
Для начала загрузим карту через подготовленный класс:

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from src.osmnx_utils import PlaceGraph

# Feasible
graph = PlaceGraph("Kazan, Russia", road_mode="bike")

# requires optimization
#graph = PlaceGraph("Saint Petersburg, Russia", road_mode="bike")

graph.plot_graph()

Сформируем задачу, выбрав известные на картах точки. Например, запросим список всех баров города и выберем половину для доставки.

In [ ]:
problem = graph.get_travelling_salesman_problem("bar pub biergarten".split(), ratio=0.5)

print(len(problem.target_nodes))

Объект *problem* содержит все необходимые данные для решения задачи.\
В последствии мы будем обращаться к графу карты города исключительно для визуализации.

In [ ]:
graph.plot_graph(problem.target_nodes)

# Generic TSP solver

Для решения задачи коммивояжера мы будем использовать методы на базе алгоритма Hill Climbing. 
Ниже представлена его обобщенная реализация (**в ней ничего менять не нужно**).

In [ ]:
from typing import List
from tqdm.auto import tqdm
import numpy as np


class GenericHillClimbSolver:
    def __init__(self, solution_pool_size=1):
        self.solution_pool_size = solution_pool_size

    def init_solutions(self, solution_size: int) -> List[np.ndarray]:
        pass

    def make_options(self, solutions: List[np.ndarray]) -> List[np.ndarray]:
        pass

    def evaluation_function(
        self, solution: np.ndarray, distance_matrix: np.ndarray, **kwargs
    ) -> float:
        pass

    def select_best_options(
        self, options: List[np.ndarray], distance_matrix: np.ndarray
    ) -> List[np.ndarray]:
        pass

    def init_iteration_parameters(self, distance_matrix):
        pass

    def main_iteration(self, best_solutions, record, distance_matrix) -> (float, List[np.ndarray]):
        pass
    
    def solve(self, distance_matrix: np.ndarray, tol_steps=0):
        self.init_iteration_parameters(distance_matrix)
        base_tol_steps=tol_steps
        best_solutions = self.init_solutions(distance_matrix.shape[0])
        assert isinstance(best_solutions, list) and len(best_solutions) > 0

        record = -np.inf #self.evaluation_function(best_solutions[0], distance_matrix)
        assert record <= 0, "We use negative values for MAX optimization"

        pbar = tqdm(total=tol_steps+1, mininterval=5, maxinterval=100, leave=True)
        while True:

            session_record, best_options = self.main_iteration(best_solutions, record, distance_matrix)

            if session_record <= record:
                tol_steps-=1
                if tol_steps % 10:
                    pbar.update(10)
                if tol_steps < 0:
                    pbar.close()
                    return best_solutions[0], record
            else:
                tol_steps=base_tol_steps
                pbar.n=0
                pbar.refresh()
                
            if session_record >= record:
                if (session_record > record):
                    pbar.set_description(f"Current score: {session_record}")
                best_solutions = best_options
                record = session_record

## Simple hill climbing

In [ ]:
def get_solution_path(solution, path_matrix):
    route = []
    for a,b in zip(solution, np.roll(solution, -1)):
        path=path_matrix[a][b]
        if len(path) == 0:
            break
        route = route[:-1] + path
    return route

def swap_positions(array, positions):
    positions = list(positions)
    array = np.copy(array)
    array[positions] = array[positions[::-1]]
    return array

def get_solution_cost(solution, distance_matrix):
    return sum(distance_matrix[a][b] for a, b in zip(solution, np.roll(solution, -1)))

Первый вариант алгоритма - стандартный Hill Climbing.

Формальное описание в терминах методов класса GenericHillClimbSolver:
1. *init_solutions* - Инициализируем начальное решение (любой порядок точек).
2. *make_options* - Выпишем все соседние варианты решений. Для задачи коммивояжера соседнее решение это просто перестановка любой пары вершин (используйте функцию *swap_positions*).
3. *select_best_options* - Отсортируем соседние решения по оптимальности (функция *get_solution_cost*) и выберем лучшие.
4. *evaluation_function* - Зафиксируем лучший вариант итерации и запомним его оценку оптимальности
5. Если вариант решения, полученный в результате шагов 2-4 лучше известного, то возвращаемся на шаг 2, иначе заканчиваем работу

**Шаг 5 реализовывать не надо** - вместо этого нужно организовать вызов методов шагов 2-4 в методе *main_iteration*, который должен возвращать оценку лучшего решения и само решение.


In [ ]:
from itertools import combinations


class SimpleHillClimb(GenericHillClimbSolver):
    def init_solutions(self, solution_size: int) -> List[np.ndarray]:
        ### TO IMPLEMENT ###
        pass
    
    def make_options(self, solutions: List[np.ndarray]) -> List[np.ndarray]:
        ### TO IMPLEMENT ###
        pass

    def evaluation_function(
        self, solution: np.ndarray, distance_matrix: np.ndarray, **kwargs
    ) -> float:
        ### TO IMPLEMENT ###
        pass

    def select_best_options(self, options, distance_matrix):
        ### TO IMPLEMENT ###
        pass
        
    def main_iteration(self, best_solutions, record, distance_matrix):
        ### TO IMPLEMENT ###
        pass

In [ ]:
solver = SimpleHillClimb()

path, cost=solver.solve(problem.distance_matrix, tol_steps=0)
path, cost

In [ ]:
graph.plot_route_animated(get_solution_path(path, problem.shortest_paths), problem.target_nodes, render_every=50)

## Stochastic hill climbing

Очевидный недостаток стандартного алгоритма Hill Climbing - локальные минимумы.
Попробуем из них выбраться, заменив выбор лучшего решения на случайный.

Для этого модифицируем метод *select_best_options*

In [ ]:
class StochasticHillClimb(SimpleHillClimb):
    def select_best_options(
        self, options: List[np.ndarray], distance_matrix: np.ndarray
    ) -> List[np.ndarray]:
        ### TO IMPLEMENT ###
        pass

In [ ]:
solver = StochasticHillClimb()

path, cost=solver.solve(problem.distance_matrix, tol_steps=10000)
path, cost

In [ ]:
graph.plot_route_animated(get_solution_path(path, problem.shortest_paths), problem.target_nodes, render_every=50)

## Simulated annealing

Случайное блуждание по соседним вариантам, хоть и может дать более оптимальное решение, все же сходится крайне медленно.

Вот и придумали алгоритм имитации отжига (simulated annealing).\
Смысл следующий. Давайте вместо перехода только в лучшие решения $S^*$ будем с некоторой вероятностью переходить в любые случайные решения $S$.

Вероятность перехода в новое решение задается следующим образом:
* $P(accept|score(S)>score(S^*)) = 1$
* $P(accept|score(S)\leq score(S^*)) = exp(\frac{-|score(S^*)-score(S)|}{T})$

где $T$ - некоторый параметр температуры распределения, контролирующий диапазон допустимых отклонений от известного оптимума.

Поскольку по мере работы алгоритма поиска мы все ближе подходим к оптимуму начинают с большой температуры $T$ и с каждой итерацией снижают градус:
$$T_{i+1}=T_i*cooling\_factor$$

Для реализации алгоритма имитации отжига достаточно:
* добавить сброс температуры в инициализацию процесса поиска *init_iteration_parameters*
* добавить механизм вероятностного перехода в основной цикл *main_iteration*

In [ ]:
class SimulatedAnnealing(StochasticHillClimb):
    def __init__(self, solution_pool_size=1, temperature=100, cooling_factor=0.999):
        super().__init__(solution_pool_size)
        self.T_0 = temperature
        self.cool = cooling_factor

    def init_iteration_parameters(self, distance_matrix):
        ### TO IMPLEMENT ###
        pass
    
    def main_iteration(self, best_solutions, record, distance_matrix):
        ### TO IMPLEMENT ###
        pass

In [ ]:
solver = SimulatedAnnealing()

path, cost=solver.solve(problem.distance_matrix, tol_steps=10000)
path, cost

In [ ]:
graph.plot_route_animated(get_solution_path(path, problem.shortest_paths), problem.target_nodes, render_every=50)

## Tabu Search

Очевидным недостатком алгоритма имитации отжига (да и других стохастических методов) - возможность повторного сэмплирования пройденных позиций.\
Простейшее решение - кэширования последних **k принятых** (по вероятностному порогу) решений.

Дополните методы **init_iteration_parameters** и адаптируйте метод **main_iteration** под механизм памяти.

Используйте готовые методы для записи, проверки и загрузки решений из памяти.

In [ ]:
import json

class TabuSearch(SimulatedAnnealing):
    def __init__(self, solution_pool_size=1, temperature=100, cooling_factor=0.999, memory_size=50):
        super().__init__(solution_pool_size, temperature=temperature, cooling_factor=cooling_factor)
        self.memory_size = memory_size

    def convert_to_memory(self, solution):
        return json.dumps(solution.tolist())

    def load_from_memory(self, memory):
        return np.array(json.loads(memory))
    
    def add_to_memory(self, solution):
        self.memory.append(self.convert_to_memory(solution))
        self.memory = self.memory[-self.memory_size:]

    def is_in_memory(self,solution):
        return convert_to_memory(solution) in self.memory
    
    def init_iteration_parameters(self, distance_matrix):
        super().init_iteration_parameters(distance_matrix)
        ### TO IMPLEMENT ###
        pass
    
    def main_iteration(self, best_solutions, record, distance_matrix):
        ### TO IMPLEMENT ###
        pass

In [ ]:
solver = TabuSearch()

path, cost=solver.solve(problem.distance_matrix, tol_steps=10000)
path, cost

In [ ]:
graph.plot_route_animated(get_solution_path(path, problem.shortest_paths), problem.target_nodes, render_every=None)

## Genetic algorithm

Чистые стохастические процессы слишком непредсказуемы, чтобы верить в и оптимальность (да и сходятся они лет 100).\
Поэтому стоит сохранять детерминированность для этапа выбора лучшего решения, как это и делают генетические алгоритмы.

По факту генетический алгоритм это разновидность классического Hill Climbing:
1. *init_solutions* - Инициализируем $k$ случайных решений.
2. *make_options* - Из всего пула выберем 2 случайных решения $a,b$ из пула и для них:
    1. Выберем случайно $0\leq h \leq len(a)$
    2. Проведем процедуру **Скрещевания** решений:
        * Построим отображения (перестановки) $\sigma_a, \sigma_b$ тривиального решения $O$ в решения $а,b$
        * Создадим новые перестановки $\sigma_1 = \sigma_a[:h]+\sigma_b[h:]$ и $\sigma_2 = \sigma_b[:h]+\sigma_a[h:]$
        * Применим их к $O$ и получим два новых решения $с_1,с_2$
    3. Проведем процедуру **Мутации** для каждого нового решения $c_i$:
        * Выберем позиции из распределения Бернулли с вероятность мутации позиции $p_mut$
        * Перемешаем мутирующие позиции в решении $с$ любым способом (можно использовать функцию *swap_positions*)
    4. Добавим новые решения к имеющемуся пулу решений и отправим его на оценку
3. *select_best_options* - Отсортируем решения по оптимальности и выберем $k$ лучших (держим популяцию под контролем).
4. *evaluation_function* - Зафиксируем лучший вариант итерации и запомним его оценку оптимальности (**Так же как в Hill Climbing**)
5. Если вариант решения, полученный в результате шагов 2-4 лучше известного, то возвращаемся на шаг 2, иначе заканчиваем работу (**Так же как в Hill Climbing**)

Таким образом, для реализации генетического алгоритма нужно перегрузить только 3 функции.

In [ ]:
import random

class GeneticAlgorithm(SimpleHillClimb):
    def __init__(self, solution_pool_size=10, mutation_rate=0.1):
        super().__init__(solution_pool_size)
        self.mut_r = mutation_rate
        
    def init_solutions(self, solution_size: int) -> List[np.ndarray]:
        ### TO IMPLEMENT ###
        pass

    def make_options(self, solutions: List[np.ndarray]) -> List[np.ndarray]:
        ### TO IMPLEMENT ###
        pass
    

In [ ]:
solver = GeneticAlgorithm(mutation_rate=0.05)

path, cost=solver.solve(problem.distance_matrix, tol_steps=10000)
path, cost

In [ ]:
graph.plot_route_animated(get_solution_path(path, problem.shortest_paths), problem.target_nodes, render_every=50)

## Ant colony optimization

Генетический алгоритм хоть и эффективный, но все ещё случайный. 
При генерации решений не используется никакая информация об оптимальности и поэтому имеем ту же проблему, что и в стохастическом Hill Climbing - можем генерировать постоянно неоптимальные решения и тем самым застревать в локальном минимуме.
Есть решения вроде инбридинга, но это для сторонников голубой крови и монархии)

Вместо шаманизма можно посмотреть на макроуровень природы - на роевой интеллект. 
Идея любого роевого интеллекта - много параллельных агентов и синхронизации их знаний на каждой итерации. Поскольку агенты начинают со случайных решений и двигаются на первых итерациях довольно хаотично, то мы сохраняем все преимущества стохастических методов, однако синхронизация позволяет сокращать хаотичное исследование в пользу организованной эксплуатации лучших решений, тем самым гарантируя, что на последних итерациях мы будем только улучшать известное.

Муравьиный алгоритм (Ant Colony Optimization) - наиболее известный вариант роевого интеллекта. В наших терминах он имеет следующую логику:
1. *init_solutions* - Для каждого агента колонии выберем случайную **точку** старта решения
2. *make_options* - для каждого агента достроим решение до конца:
    1. Пока все точки не вошли в решение $S_{cur}$:
       * Отмечаем последнюю точку $i$ текущего решения $S_{cur}$ как посещенную
       * Для всех точек $j$ определим вероятность
         $$P_{ij}=\frac{\tau_{ij}^\alpha + D_{ij}^{-\beta}}{\sum_{j}{\tau_{ij}^\alpha + D_{ij}^{-\beta}}}$$
         где $\tau$ - феромоны, привлекательность ребра с точки зрения известной рою оптимальности, $D$ - матрица длин кратчайших путей между точками (веса ребер), $\beta\geq 1$ и $\alpha \geq 0$ - гиперпараметры
       * В качестве следующей точки $next$ выберем такую, что $P_{i,next}=\max_{j}{P_{ij}}$
       * Добавим точку $next$ в решение $S_{cur}$ и закончим итерацию
    2. Для всех ребер $ij$ решения $S_{cur}$ запишем обновление для матрицы феромонов
       $$\Delta\tau_{ij}=\Delta\tau_{ij} + \frac{\delta}{|S_{cur}|}$$
       где $\delta$ - количество феромонов, которое агент может распределить по всему пути
3. Обновим матрицу феромонов для точек по формуле:
   $$\tau_{ij}=(1-\rho)\tau_{ij}+\Delta\tau_{ij}$$
   где $0\leq \rho \leq 1$ - скорость испарения феромонов
4. *select_best_options* - Отсортируем соседние решения по оптимальности и выберем лучшие. (**Так же как в Hill Climbing**)
5. *evaluation_function* - Зафиксируем лучший вариант итерации и запомним его оценку оптимальности (**Так же как в Hill Climbing**)
6. Если вариант решения, полученный в результате шагов 1-4 лучше известного, то возвращаемся на **шаг 1**, иначе заканчиваем работу (**НА КАЖДОЙ ИТЕРАЦИИ НУЖНО ИНИЦИАЛИЗИРОВАТЬ РЕШЕНИЯ АГЕНТОВ**)

Как можно заметить в отличие от Hill Climbing, в муравьином алгоритме лучшее решение не используется явно, а вместо этого записывается в матрицу феромонов во взвешенной форме, что позволяет агентам рассматривать его модификации в наиболее слабых (по весу) местах.  


In [ ]:
class AntColony(SimpleHillClimb):
    def __init__(self, colony_size=10, alpha=0.5, beta=1.0, delta=1.0, rho=0.9):
        super().__init__(solution_pool_size=colony_size)
        self.alpha = alpha
        self.beta = beta
        self.rho = rho
        self.delta = delta

    def init_iteration_parameters(self, distance_matrix):
        self.pheromones = np.zeros_like(distance_matrix)

    def init_solutions(self, solution_size: int) -> List[np.ndarray]:
        ### TO IMPLEMENT ###
        pass

    def make_options(self, solutions: List[np.ndarray], distance_matrix) -> List[np.ndarray]:
        ### TO IMPLEMENT ###
        pass
            
    
    def main_iteration(self, best_solutions, record, distance_matrix):
        ### TO IMPLEMENT ###
        pass

In [ ]:
solver = AntColony()

path, cost=solver.solve(problem.distance_matrix, tol_steps=1000)
path, cost

In [ ]:
graph.plot_route_animated(get_solution_path(path, problem.shortest_paths), problem.target_nodes, render_every=50)